# 第八章 机器学习心理学应用

In [ ]:
"""
本代码为《Python心理学应用》一书第八章机器学习的全部代码
建议结合书本内容进行实操练习，以增进理解
王利刚，wanglg@psych.ac.cn
魏楚光，weicg@psych.ac.cn 
8-Dec-2024
"""

### 代码8.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#(1)定义计算MSE的函数
def calculate_MSE(x,y,w):
    y_pre=w[0]*x+w[1]
    MSE=np.mean((y_pre-y)**2)
    return MSE
#(2)计算均方误差对w0和w1的偏导数
def gradient_MSE(x,y,w):
    y_pre=w[0]*x+w[1]
    d_w0=2*np.mean((y_pre-y)*x)
    d_w1=2*np.mean(y_pre-y)
    return d_w0,d_w1
#(3)参数更新函数
def updata_Parameter(x,y,tau_max):
    alpha = 0.000001 # 学习率
    tau_max = tau_max # 重复的最大次数
    eps = 0.1 # 停止重复的梯度绝对值的阈值
    w_history = np.zeros([tau_max, 2]) #用于存储参数变化的空白数组
    MSE_history=[] #存储更新参数过程中对应的MSE值
    w_history[0,:] = [2, 278]# 设置初始参数
    for tau in range( 1, tau_max):
        MSE = calculate_MSE(x, y, w_history[tau - 1])
        MSE_history.append(MSE)
        dMSE = gradient_MSE(x, y, w_history[tau - 1])
        w_history[ tau, 0] = w_history[tau - 1, 0] - alpha * dMSE[ 0]  #跟新参数w0
        w_history[ tau, 1] = w_history[ tau - 1, 1] - alpha * dMSE[ 1] #跟新参数w1
        if max( np. absolute( dMSE)) < eps: # 结束判断
           break
    w0 = w_history[ tau_max-1, 0]
    w1 = w_history[ tau_max-1, 1]
    w_history = w_history.reshape((2,tau_max))
    return w0, w1, dMSE, w_history,MSE_history
#(4)运用梯度法降低法求参数，并绘图
#生成心理健康水平和心理需求满足度得分数据
np.random.seed(123)
X=89+170*np.random.rand(21)#模拟生成心理需求满足度量表得分
Y=300-180*np.exp(-0.02*X)+10*np.random.rand(21) #模拟生成心理健康量表得分
 #调用梯度法更新参数
W0,W1,dMSE,w_history,MSE_history=updata_Parameter(X,Y,tau_max=100)
print("迭代100次获得的系数分别是：w0={0},w2={1},MSE={2}".format(W0,W1,MSE_history[-1]))
#绘图部分
fig,ax=plt.subplots(1,2)
x=np.linspace(100,260,1000)#生成一个等差数列
Y_pre=W0*x+W1 #使用等差数量形成对于的Y值，用于绘制直线
#绘制散点图和拟合直线
ax[0]. plot( X, Y, marker = 'o', linestyle = 'None', markeredgecolor = 'black', color = 'gray')
# sns.regplot(x= X,y=Y ,color="gray", line_kws={'color': 'gray', 'alpha': 0.7})
ax[0].plot(x,Y_pre)
ax[0].set_xlabel("需求满足度",fontproperties='SimHei')
ax[0].set_ylabel("心理健康",fontproperties='SimHei')
ax[0].set_title("梯度法参数的拟合情况",fontproperties='SimHei')
#绘制在迭代过程中MSE的变化情况
ax[1].plot(MSE_history)
ax[1].set_xlabel("迭代次数",fontproperties='SimHei')
ax[1].set_ylabel("MSE")
ax[1].set_title("梯度调参数的均方误差变化",fontproperties='SimHei')
plt.show()

### 代码8.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def entropy(p): #计算信息熵的函数     
    if p == 0 or p == 1:
        return 0
    else:   
        return -p * np.log2(p) - (1 - p) * np.log2(1 - p)
#计算不同概率下，事件的信息熵
probabilities = np.arange(0, 1.01, 0.01)#生成概率值从0到1，间隔0.01
entropies = [entropy(p) for p in probabilities]#生成概率值对应的信息熵
#绘制信息熵与事件概率的函数
plt.plot(probabilities, entropies)
plt.xlabel('事件发生的概率',fontproperties='SimHei')
plt.ylabel('事件信息熵',fontproperties='SimHei')
plt.title('事件发生概率与事件信息熵的关系',fontproperties='SimHei')
plt.grid(True)
plt.show()

### 代码8.3

In [ ]:
import pandas as pd
import numpy as np
#(1)构建ID3的核心函数
#定义计算信息熵的函数
def entropy(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    entropy_value = -np.sum(probabilities * np.log2(probabilities))
    return entropy_value
#定义计算条件信息熵的函数
def conditional_entropy(data, condition_col, target_col):
    unique_conditions = data[condition_col].unique()
    total_entropy = 0
    for condition in unique_conditions:
        subset = data[data[condition_col] == condition]
        subset_prob = len(subset) / len(data)
        subset_entropy = entropy(subset[target_col])
        total_entropy += subset_prob * subset_entropy
    return total_entropy
#定义计算信息增益的函数
def information_gain(data, feature_col, target_col):
    original_entropy = entropy(data[target_col])
    cond_entropy = conditional_entropy(data, feature_col, target_col)
    return original_entropy - cond_entropy
#选择最佳分裂特征的函数
def choose_best_feature(data, target_col):
    features = data.columns.drop(target_col)
    best_feature = None
    max_info_gain = -1
    for feature in features:
        ig = information_gain(data, feature, target_col)
        if ig > max_info_gain:
            max_info_gain = ig
            best_feature = feature
    return best_feature
#多数表决函数，用于确定叶节点的类别
def majority_vote(labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    return unique_labels[np.argmax(counts)]
#递归构建决策树
def build_decision_tree(data, target_col, max_depth=None, min_samples_split=2, current_depth=0):
    # 终止条件 1: 所有样本属于同一类别
    if len(np.unique(data[target_col])) == 1:
        return np.unique(data[target_col])[0]
    # 终止条件 2: 没有可用于分裂的特征
    if len(data.columns) == 1:
        return majority_vote(data[target_col])
    # 终止条件 3: 达到预设的最大深度
    if max_depth is not None and current_depth >= max_depth:
        return majority_vote(data[target_col])
    # 终止条件 4: 节点中的样本数量少于预设的最小值
    if len(data) < min_samples_split:
        return majority_vote(data[target_col])
    #选择最佳分裂特征
    best_feature = choose_best_feature(data, target_col)
    tree = {best_feature: {}}
    # 根据最佳分裂特征的不同取值划分数据集
    unique_values = data[best_feature].unique()
    for value in unique_values:
        subset = data[data[best_feature] == value].drop(columns=[best_feature])
        # 递归构建子树，增加当前深度
        subtree = build_decision_tree(subset, target_col, max_depth, min_samples_split, current_depth + 1)
        tree[best_feature][value] = subtree
    return tree
#利用决策树进行预测
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree
    feature = list(tree.keys())[0]
    value = sample[feature]
    subtree = tree[feature].get(value)
    if subtree is None:
        # 如果测试样本的特征值在训练数据中未出现，可根据实际情况处理，这里简单返回多数类别
        all_labels = []
        for sub in tree[feature].values():
            if not isinstance(sub, dict):
                all_labels.append(sub)
        return majority_vote(all_labels)
    return predict(subtree, sample)
#（2）准备数据包含标签和特征数值
label_data=[0]*88+[1]*18 #是否心理疾患，总数是106人
feature1=[1]*68+[0]*20+[1]*6+[0]*12  #是否有上家企业的推荐信
feature2=[0]*84+[1]*4+[0]*12+ [1]*6 #是否存在持续一年没有固定工作
data = pd.DataFrame({ "是否心理疾患": label_data, "是否有推荐信":feature1, "是否1年无工作":feature2})
print("标签与特征数据库： ",data)
#构建决策树，设置最大深度为 3，最小样本分割数为 2
decision_tree = build_decision_tree(data, "是否心理疾患", max_depth=3, min_samples_split=2)
print("决策树结构:")
print(decision_tree)
#待预测样本
sample = { "是否有推荐信": 0, "是否1年无工作": 1}
#进行预测
prediction = predict(decision_tree, sample)
print(f"预测结果: {'是' if prediction == 1 else '否'} 患有心理疾患")

### 代码8.4

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
#（1）模拟数据
label_data = [0] * 88 + [1] * 18 # 是否心理疾患，总数是106人
feature1 = [1] * 68 + [0] * 20 + [1] * 6 + [0] * 12 # 是否有上家企业的推荐信
feature2 = [0] * 84 + [1] * 4 + [0] * 12 + [1] * 6 # 是否存在持续一年没有固定工作
data = pd.DataFrame({"是否心理疾患": label_data,"是否有推荐信": feature1,"是否1年无工作": feature2})
X = data.drop("是否心理疾患", axis=1) #这是分裂特征数据
y = data["是否心理疾患"]  #这是标签数据
#（2）划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#（3）创建决策树分类器
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)#训练模型
#（4）使用训练好的模型进行预测并评估模型
y_pred = clf.predict(X_test)#用输入测试数据，生成测试结果
accuracy = accuracy_score(y_test, y_pred)#评估模型分类准确性
print(f"模型的准确率: {accuracy:.2f}")
#（5）以图形方式展示决策树
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题
plt.figure(figsize=(12, 8))
plot_tree(clf, feature_names=X.columns, class_names=['无心理疾患', '有心理疾患'], filled=True)
plt.show()
#（6）以文本形式输出决策树结构
tree_text = export_text(clf, feature_names=list(X.columns))
print("决策树文本结构： ",tree_text)

### 代码8.5

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
#（1）模拟生成数据
np.random.seed(42)
n_samples =500
# 生成人格各维度得分
extroversion = np.random.rand(n_samples) * 10 # 外向性
agreeableness = np.random.rand(n_samples) * 10 # 宜人性
conscientiousness = np.random.rand(n_samples) * 10 # 责任心
emotional_stability = np.random.rand(n_samples) * 10 # 情绪稳定性
openness = np.random.rand(n_samples) * 10 # 开放性
# 合并人格各维度得分作为特征矩阵
X = np.column_stack((extroversion, agreeableness, conscientiousness, emotional_stability, openness))
#模拟生成心理健康得分（目标变量）
y = 0.5 * extroversion + 0.3 * agreeableness + 0.4 * conscientiousness + 0.6 * emotional_stability + 0.2 * openness + np.random.normal(0, 1, n_samples)#假设心理健康得分与人格维度得分存在一定的线性关系，并加入一些噪声
#（2）训练阶段
#① 数据准备
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  #划分训练集和测试集
# ② 确定参数
param_grid = {'n_estimators': [50, 100, 150],'max_features': ['sqrt', 'log2']}
# 创建随机森林回归器实例
rf = RandomForestRegressor(random_state=42, oob_score=True)
# 创建网格搜索实例，使用 5 折交叉验证
grid_search = GridSearchCV(rf, param_grid, cv=5)
# 在训练集上进行网格搜索
grid_search.fit(X_train, y_train)
# 获取最优参数
best_params = grid_search.best_params_
n_estimators = best_params['n_estimators']
max_features_str = best_params['max_features']
if max_features_str =='sqrt':
    max_features =int(np.sqrt(X_train.shape[1]))
elif max_features_str == 'log2':
    max_features =int(np.log2(X_train.shape[1]))#
#③ 自助采样及并构建决策树，后续步骤在 RandomForestRegressor 中自动完成
rf = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features, random_state=42, oob_score=True)#
rf.fit(X_train, y_train)
#（3）预测阶段
# ① 输入待预测样本，集成预测结果在 predict 方法中自动完成
y_pred = rf.predict(X_test)
# ②模型评估
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
oob_score = rf.oob_score_
print(f"最优参数 - 决策树数量: {n_estimators}")
print(f"最优参数 - 每个节点分裂时随机选择的特征数量: {max_features}")
print(f"均方误差 (MSE): {mse:.2f}")
print(f"均方根误差 (RMSE): {rmse:.2f}")
print(f"平均绝对误差 (MAE): {mae:.2f}")
print(f"袋外数据得分 (OOB Score): {oob_score:.2f}")#
#③特征重要性
feature_importances = rf.feature_importances_
feature_names = ['外向性', '宜人性', '责任心', '情绪稳定性', '开放性']
#④可视化预测结果和特征重要性
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
plt.figure(figsize=(18, 6))
#子图 1：预测值 vs 真实值散点图
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred,label='预测值与真实值关系')
plt.xlabel('真实值')
plt.ylabel('预测值')
plt.title('预测值 vs 真实值')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--', label='理想预测线')#绘制对角线
plt.legend()
# 子图 2：特征重要性饼图
plt.subplot(1, 2, 2)
#凸显最重要特征
index_most_important=np.argmax(feature_importances)
explode = [0, 0, 0, 0, 0]
explode[index_most_important]=0.1 # 突出显示最重要的特征切片
wedges, texts, autotexts = plt.pie(feature_importances, explode=explode, labels=feature_names,autopct='%1.1f%%', shadow=True, startangle=140)
#设置文本属性
plt.setp(autotexts, size=10, weight="bold")
plt.title('特征重要性')
plt.tight_layout()
plt.show()

### 代码8.6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#(1)模拟生成特征数据与标签数据
np.random.seed(0)
X = np.vstack([np.random.randn(20, 2)*5 -[10, 10], np.random.randn(20, 2)*5 + [10, 10]])+[30,30]#模拟生成特征数据
X=np.round(X) #数据取整
y = np.hstack([np.zeros(20), np.ones(20)]) #模拟生成标签数据
data=pd.DataFrame() #创建一个空的dataFrame
data[["情绪稳定性","外向性"]]=X #存储特征数据到datafrme
data["心理健康"]=y #存储标签数据到datafrme
print("人格与心理健康数据：",data)
data.to_excel("人格与心理健康数据.xlsx",index=False) #导出数据到excel中，方便后续重复使用
#(2)绘制两个特征数据的散点图，并以心理健康标签数据标注散点颜色
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
# 根据心理健康标签将数据分为两类
class_0 = data[data["心理健康"] == 0]
class_1 = data[data["心理健康"] == 1]
#绘制散点图
plt.scatter(class_0["情绪稳定性"], class_0["外向性"], c='blue', label='心理健康类别 0')
plt.scatter(class_1["情绪稳定性"], class_1["外向性"], c='red', label='心理健康类别 1')
plt.title('情绪稳定性和外向性的散点图')
plt.xlabel('情绪稳定性')
plt.ylabel('外向性')
plt.legend()
plt.show()

### 代码8.7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC
#（1）导入数据库数据
data=pd.read_excel("人格与心理健康数据.xlsx")
X=data[["情绪稳定性","外向性"]].values
y=data["心理健康"].values
#（2）创建并训练 SVM 分类器
clf = SVC(kernel='linear')
clf.fit(X, y)
#（3）获取超平面参数
w = clf.coef_[0]
b = clf.intercept_[0]
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
xx = np.linspace(x_min, x_max)
yy = -(w[0] * xx + b) / w[1]
#计算间隔距离 d
d = 2 / np.linalg.norm(w)
#计算间隔边界
yy_up = -(w[0] * xx + b + 1) / w[1]
yy_down = -(w[0] * xx + b - 1) / w[1]
#获取支持向量
support_vectors = clf.support_vectors_
#（4）可视化最优超平面及分隔空间
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
plt.scatter(X[y == 0][:, 0], X[y == 0][:, 1], c='b', label='心理健康类别0')
plt.scatter(X[y ==1][:, 0], X[y == 1][:, 1], c='r', label='心理健康类别1')
plt.plot(xx, yy, 'g--', label='最优超平面')
plt.plot(xx, yy_up, 'k--', label='分隔空间上边界')
plt.plot(xx, yy_down, 'k--', label='分隔空间下边界')
#标记支持向量
plt.scatter(support_vectors[:, 0], support_vectors[:, 1], s=100, edgecolors='y', facecolors='none', label='支持向量')
#标注间隔距离 d
mid_point_x = (x_min + x_max) / 2
mid_point_y = -(w[0] * mid_point_x + b) / w[1]
plt.annotate(f'd = {d:.2f}', xy=(mid_point_x, mid_point_y), xytext=(mid_point_x+1.5, mid_point_y-4),
arrowprops=dict(facecolor='black', shrink=0.05))
plt.xlabel('情绪稳定性')
plt.ylabel('外向性')
plt.title('基于两个特征变量的支持向量机分类')
plt.ylim(5,60)
plt.legend(loc="upper left")
plt.show()

### 代码8.8

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
#（1）生成模拟数据
np.random.seed(0)
X = np.sort(5 * np.random.rand(40, 1)+30, axis=0) # 模拟生成40个外向性得分数据
y = np.sin(X).ravel()+20 # 模拟生成40个心理健康得分数据
y[::5] += 3 * (0.5 - np.random.rand(8)) # 给数据增加点噪声和随机变化
#（2）确定epsilon参数
param_grid = {'epsilon': np.linspace(0.1, 1, 10)}# 定义要搜索的epsilon参数范围
#创建SVR模型
svr = SVR(kernel='rbf', C=100, gamma=0.1)
#使用GridSearchCV进行网格搜索和交叉验证
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)
#获取最优的epsilon参数
best_epsilon = grid_search.best_params_['epsilon']
print(f"最优的epsilon参数: {best_epsilon:.4f}")
#（3）拟合SVR模型
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=best_epsilon)
svr_rbf.fit(X, y)
# （4）计算管道空间的上下边界
y_rbf = svr_rbf.predict(X)
y_upper = y_rbf + best_epsilon
y_lower = y_rbf - best_epsilon
# （5）计算评价指标
mse = mean_squared_error(y, y_rbf)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_rbf)
#打印评价指标
print(f"均方误差 (MSE): {mse:.4f}")
print(f"均方根误差 (RMSE): {rmse:.4f}")
print(f"决定系数 (R的平方): {r2:.4f}")
# (6)可视化拟合结果
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
plt.scatter(X, y, c='k', label='数据点')
plt.plot(X, y_rbf, c='g', label='最优超平面', linewidth=2)
plt.fill_between(X.flatten(), y_lower, y_upper, color='gray', alpha=0.2, label='管道空间')
#在图中添加评价指标
textstr = f"均方误差 (MSE): {mse:.4f}均方根误差 (RMSE): {rmse:.4f}决定系数 (R的平方): {r2:.4f}"
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes, fontsize=10,verticalalignment='top', bbox=props)
plt.xlabel('外向性')
plt.ylabel('心理健康')
plt.title('支持向量机回归任务')
plt.legend()
plt.show()

### 代码8.9

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#(1)欧式距离计算函数
def euclidean_distance_numpy(x, y):
    x = np.array(x)
    y = np.array(y)
    return np.linalg.norm(x - y)
#(2)准备数据
data={"你":[170,60],"大毛":[180,60],"二毛":[170,68],"小明":[167,64]}
#计算与你各个同学之间的欧式距离：
for name in ["大毛","二毛","小明"]:
    distance = euclidean_distance_numpy(data["你"], data[name])
    print(name+"与你的欧氏距离为:", distance)
#(3)把数据放置到二维空间中目测
# 提取每个学生的坐标
your_coords = data["你"]
big_hair_coords = data["大毛"]
second_hair_coords = data["二毛"]
xiaoming_coords = data["小明"]
#设置图片可以显示中文
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
colors=["red","blue","green","purple"]
labels=["你","大毛","二毛","小明"]
#创建散点图，用不同颜色表示不同学生
for i in range(len(data)):
    label=labels[i]
    color=colors[i]
    coords=data[label]
    plt.scatter(coords[0], coords[1], color=color, label=label)
    if label!="你":
        your_coords = data["你"]
        plt.plot([your_coords[0], coords[0]], [your_coords[1], coords[1]],color=color, label=label+'与你的距离')
#添加标题和坐标轴标签
plt.title('同学数据点及与你的距离')
plt.xlabel('身高')
plt.ylabel('体重')
plt.legend()
plt.show()

### 代码8.10

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def kmeans(X, K, max_iterations=100):  # 随机初始化K个质心
    n_samples, n_features = X.shape    # 从数据集中随机选择K个样本作为初始质心
    centroids = X[np.random.choice(n_samples, K, replace=False)]
    #创建一个存放迭代过程中质心和簇标签的字典
    centroids_labels_history={}
    for i in range(max_iterations):
        # 分配样本到最近的质心
        # 用于存储每个样本所属的簇标签
        labels = []
        for sample in X:
            # 计算当前样本到每个质心的距离
            distances = [np.linalg.norm(sample - centroid) for centroid in centroids]
            # 找到距离最近的质心的索引
 
            label = np.argmin(distances)
            labels.append(label)
        labels = np.array(labels)
        #把数据质心和对应的簇成员标签存放进字典
        centroids_labels_history[i]=(centroids,labels)
        # 更新质心
        new_centroids = []
        for k in range(K):
            # 找到属于当前簇的所有样本
            cluster_samples = X[labels == k]
            if len(cluster_samples) > 0:
                # 计算当前簇内所有样本的均值作为新的质心
                new_centroid = np.mean(cluster_samples, axis=0)
                new_centroids.append(new_centroid)
            else:                # 如果某个簇为空，则重新随机选择一个质心
                new_centroids.append(X[np.random.choice(n_samples)])
        new_centroids = np.array(new_centroids)
        # 判断质心是否不再变化，如果是则提前结束迭代
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids
    return centroids_labels_history
#生成班级60人的体重和身高数据
np.random.seed(42)
X = np.vstack([np.random.normal(loc=[55,160], scale=5, size=(20, 2)),
                    np.random.normal(loc=[65,170], scale=5, size=(20, 2)),
                    np.random.normal(loc=[60,180], scale=5, size=(20, 2))
                    ])
#设置聚类的簇数
K = 3
#调用K - 均值聚类函数
centroids_labels_history = kmeans(X, K)
iterations=len(centroids_labels_history)#最终迭代次数
print("最终聚类结果的三个质心分别是：",centroids_labels_history[iterations-1][0])
#绘制迭代过程中的质心和分类簇变化图
n_col=3#设置多图的列数
n_row, remainder=divmod(iterations,n_col)#计算多图的行数
if remainder>0:#如果有余数，行数加1
    n_row+=1
plt.rcParams['font.sans-serif'] = ['SimHei'] #可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
for key,value in centroids_labels_history.items():#遍历聚类迭代历史数据
    plt.subplot(n_row,n_col,key+1)#在n_row行和n_col列的网格中绘制第key+1个子图
    centroids,labels=value#将第key次质心和簇标签读取出来
    plt.scatter(X[:, 1], X[:, 0], c=labels, cmap='viridis')#绘制散点，并根据簇标签分类图形
    plt.scatter(centroids[:, 1], centroids[:, 0], marker='X', s=100, c='red') #绘制三个质心
    quotient, remainder = divmod(key, n_col) #判断绘图位置    
    if quotient==n_row-1:#最后一行显示x轴的label
        plt.xlabel('体重')
    if remainder==0:#左边第一列显示y轴的标签
        plt.ylabel('身高')
plt.show()

### 代码8.11

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
# （1）#生成班级60人的体重和身高数据
np.random.seed(42)
X = np.vstack([np.random.normal(loc=[55, 160], scale=5, size=(20, 2)), 
             np.random.normal(loc=[65, 170], scale=5, size=(20, 2)),
              np.random.normal(loc=[60, 180], scale=5, size=(20, 2))
              ])
#（2）定义聚类算法
algorithms = {"K -均值聚类（KMeans）": KMeans(n_clusters=3, random_state=42),                                                                                                                                     "小批量 K - 均值算法（MiniBatchKMeans）": MiniBatchKMeans(n_clusters=3, random_state=42),"凝聚式层次聚类（AgglomerativeClustering）": AgglomerativeClustering(n_clusters=3),"密度空间聚类应用算法（DBSCAN）": DBSCAN(eps=3, min_samples=5),"高斯混合模型聚类（GaussianMixture）": GaussianMixture(n_components=3, random_state=42), "谱聚类（SpectralClustering）": SpectralClustering(n_clusters=3, random_state=42) }
#（3）遍历所有的距离算法，并将聚类结果可视化
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei'
n_algorithms = len(algorithms)
n_rows =2
n_cols = 3
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 10))
#存储每个算法的评价指标结果
evaluation_results = {}
#遍历每个算法进行聚类和可视化 
for i, (name, algorithm) in enumerate(algorithms.items()):
    row = i // n_cols
    col = i % n_cols
    if name == "高斯混合模型聚类（GaussianMixture）":
        labels = algorithm.fit_predict(X)
    else:
        algorithm.fit(X)
        labels = algorithm.labels_
    #计算评价指标 
    try:
        silhouette = silhouette_score(X, labels)
        calinski_harabasz = calinski_harabasz_score(X, labels)
        davies_bouldin = davies_bouldin_score(X, labels)
        evaluation_results[name] = {'Silhouette Score': silhouette,'Calinski - Harabasz Score': calinski_harabasz,'Davies - Bouldin Score': davies_bouldin}
    except:
    #处理可能出现的异常，例如 DBSCAN 可能只有一个簇，导致某些指标无法计算
        evaluation_results[name] = {'Silhouette Score': None,'Calinski - Harabasz Score': None,'Davies - Bouldin Score': None}
    axes[row, col].scatter(X[:, 1], X[:, 0], c=labels, cmap='viridis')
    axes[row, col].set_title(name)
    axes[row, col].set_xlabel('身高')
    axes[row, col].set_ylabel('体重')
plt.tight_layout()
plt.show()
#（4）提取评价指标数据
algorithm_names = list(evaluation_results.keys())
silhouette_scores = [evaluation_results[name]['Silhouette Score'] for name in algorithm_names]
calinski_harabasz_scores = [evaluation_results[name]['Calinski - Harabasz Score'] for name in algorithm_names]
davies_bouldin_scores = [evaluation_results[name]['Davies - Bouldin Score'] for name in algorithm_names]
#（5）创建新的图形来展示评价指标
fig, axs = plt.subplots(3, 1, figsize=(10, 15))
#定义三种不同的颜色用于区分三个子图的柱子
colors = ['skyblue', 'lightgreen', 'orange']
metrics = ['Silhouette Score', 'Calinski - Harabasz Score', 'Davies - Bouldin Score']
#绘制轮廓系数横向柱状图
y_pos = np.arange(len(algorithm_names))
axs[0].barh(y_pos, silhouette_scores, color=colors[0])
axs[0].set_yticks(y_pos)
axs[0].set_yticklabels(algorithm_names)
#绘制 Calinski - Harabasz 指数横向柱状图
axs[1].barh(y_pos, calinski_harabasz_scores, color=colors[1])
axs[1].set_yticks(y_pos)
axs[1].set_yticklabels(algorithm_names)
#绘制 Davies - Bouldin 指数横向柱状图
axs[2].barh(y_pos, davies_bouldin_scores, color=colors[2])
axs[2].set_yticks(y_pos)
axs[2].set_yticklabels(algorithm_names)
#创建图例
handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in colors]
fig.legend(handles, metrics, loc='upper right')
plt.tight_layout()
plt.show()

### 代码8.12

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#模拟身高和体重数据
np.random.seed(42)
#假设身高和体重存在一定的线性关系，这里模拟100个样本
n_samples = 50
height_mean = 170
height_std = 10

weight_mean = 70
weight_std = 10
# 生成身高数据
height = np.random.normal(height_mean, height_std, n_samples)
#体重和身高有一定的正相关关系
weight = 0.5 * height + np.random.normal(weight_mean - 0.5 * height_mean, weight_std, n_samples)
#组合身高和体重数据为二维数组
data = np.column_stack((height, weight))
#计算变量方差
print("身高方差:",np.var(height))
print("体重方差:",np.var(weight))
print("总方差:",np.var(weight)+np.var(height))
#初始化角度和方差列表
angles = np.arange(0, 181, 1)
variances = []
#遍历每个角度
for angle in angles:
# 将角度转换为弧度
    rad_angle = np.radians(angle)
# 定义旋转矩阵
    rotation_matrix = np.array([[np.cos(rad_angle), -np.sin(rad_angle)],[np.sin(rad_angle), np.cos(rad_angle)]])
# 旋转坐标轴    
    rotated_data = np.dot(data, rotation_matrix)
# 计算投影到第一维的方差 
    projected_data = rotated_data[:, 0]
    variance = np.var(projected_data)
    variances.append(variance)
#获取方差最大值和相应的直线角度
max_index=np.argmax(np.array(variances))
print("形成最大方差的角度是：",angles[max_index])
print("新的一维数据方差最大值是：",variances[max_index])
min_index=np.argmin(np.array(variances))
print("形成最小方差的角度是：",angles[min_index])
print("新的一维数据方差最小值是：",variances[min_index])
# 绘制方差随角度变化的曲线
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.plot(angles, variances)
plt.xlabel('直线角度')
plt.ylabel('投影数据点的方差')
plt.title('投影直线旋转角度与降维后数据方差的关系')
plt.grid(True)
plt.show()

### 代码8.13

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA, FactorAnalysis, KernelPCA
from sklearn.manifold import LocallyLinearEmbedding, TSNE
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from scipy.stats import entropy
#（1）生成15维度的模拟数据
np.random.seed(42)
n_samples =200
n_features = 15

X = np.random.randn(n_samples, n_features)
#（2）定义降维方法
methods = {'PCA': PCA(n_components=3), 
          'Factor Analysis': FactorAnalysis(n_components=3),
          'Kernel PCA': KernelPCA(n_components=3, kernel='rbf'),
          'LLE': LocallyLinearEmbedding(n_components=3),
          't-SNE': TSNE(n_components=3)} 
#存储降维结果和评价指标
results = {}
silhouette_scores = {}
entropy_scores = {}
#颜色列表，用于区分不同评价指标的条形图
colors =['skyblue', 'lightgreen']
#(3)进行降维并计算评价指标 
for name, method in methods.items():
    X_transformed = method.fit_transform(X)#输入数据进行训练
    results[name] = X_transformed #训练结果存储到字典
    #计算轮廓系数
    kmeans = KMeans(n_clusters=2, random_state=42)
    labels = kmeans.fit_predict(X_transformed)
    silhouette_scores[name] = silhouette_score(X_transformed, labels)
    #计算信息熵 
    hist, _ = np.histogramdd(X_transformed, bins=10)
    prob = hist / hist.sum()
    entropy_scores[name] = entropy(prob.flatten())
#(4)可视化评价效果
plt.rcParams['font.sans-serif'] = ['SimHei'] #设置中文显示问题
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题
fig, axes = plt.subplots(2, 1, figsize=(8, 12))
#绘制轮廓系数的横向条形图
axes[0].barh(list(silhouette_scores.keys()), list(silhouette_scores.values()), color=colors[0], label='轮廓系数')
# 绘制信息熵的横向条形图
axes[1].barh(list(entropy_scores.keys()), list(entropy_scores.values()), color=colors[1], label='信息熵')
# 设置总图例
lines, labels = [], []
for ax in fig.axes:
    axLine, axLabel = ax.get_legend_handles_labels()
    lines.extend(axLine)
    labels.extend(axLabel)
fig.legend(lines, labels, loc='upper right')
plt.show()
#(5)获取降维各因子与原始维度的系数
np.set_printoptions(precision=2) # 设置打印选项，保留两位小数
def compontes_sort(arr):
    arr=np.abs(arr.T)
    m,n=arr.shape
    columns=["成分"+str(i+1) for i in range(n)]
    index=["特征"+str(i) for i in range(m)]
    df=pd.DataFrame(data=arr,columns=columns,index=index)
    max_column_names = df.idxmax(axis=1) #获取每行最大值的列名
    df_list=[]
    for column in df.columns:
        mask=max_column_names==column
        indices = max_column_names.index[mask]
        new_df=df.loc[indices].copy(deep=True)
        new_df=new_df.sort_values(by=column,ascending= False)
        df_list.append(new_df)
    df=pd.concat(df_list)
    return df
pca = methods['PCA']
pca_components = pca.components_
pca_components=compontes_sort(pca_components)
print("主成分分析系数表：",pca_components)
fa = methods['Factor Analysis']
fa_components = fa.components_
fa_components=compontes_sort(fa_components)
print("因子分析系数表：",fa_components)

### 代码8.14

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#（1）创建感知机的类
class Perceptron:
    def __init__(self, learning_rate=0.1, max_iter=10000):   
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = None #用于存储权重系数的动态属性
        self.bias = None #用于存储阈限值的动态属性
        self.history = {} #用于存储历史数值的动态属性
    def fit(self, X, y): 
        n_samples, n_features = X.shape
        #初始化权重和偏置为0 
        self.weights = np.zeros(n_features)
        self.bias = 0 
        for iteration in range(self.max_iter):
            misclassified =False
            for i in range(n_samples):
                #对输入与权重系数乘机加和，然后与阈限self.bias比
                linear_output = np.dot(X[i], self.weights) - self.bias
                prediction =1 if linear_output >= 0 else 0
                #如果预测错误，则更新权重和偏置 
                if prediction != y[i]:
                    self.weights += self.learning_rate * (y[i] - prediction) * X[i]
                    self.bias += -self.learning_rate * (y[i]-prediction)
                    misclassified =True 
            self.history[iteration] = (self.weights.copy(), self.bias)#将历史数据存入字典
            #如果没有错误分类的样本，则提前结束
            if not misclassified:
                break
    def predict(self, X):  
        linear_output = np.dot(X, self.weights) - self.bias
        return np.where(linear_output >= 0, 1, 0)
#（2）生成的等待分类的二维数据并创建感知机模型实例并完成训练
np.random.seed(0)
X = np.vstack([np.random.randn(20, 2) * 5 - [10, 10], np.random.randn(20, 2) * 5 + [10, 10]]) + [30, 30]
X = np.round(X)
y = np.hstack([np.zeros(20), np.ones(20)]) # 模拟生成标签数据
perceptron = Perceptron(learning_rate=0.1, max_iter=10000) #创建实例
perceptron.fit(X, y) #训练模型
#（3）调用训练好的模型进行预测
predictions = perceptron.predict(X)
print("预测结果:", predictions)
print("最终权重:", perceptron.weights)
print("最终阈值:", perceptron.bias)
print("总迭代次数:",len(perceptron.history))
#（4）可视化感知机学习过程
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei' 
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题
#绘制被分类数据的散点图
plt.scatter(X[y == 0][:, 0], X[y == 0][:, 1], c='b', label='类别0')
plt.scatter(X[y ==1][:, 0], X[y == 1][:, 1], c='r', label='类别1')
#获取 x 轴范围
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
#选择要绘制直线的迭代次数值
iterations=len(perceptron.history) #根据存放的历史数据长度，计算实际迭代次数
steps,remainder = divmod(iterations, 5) #选择历史迭代中的其中五次来绘图
keys=[key for key in np.arange(0,iterations,steps)] #获取还要绘图的5个超平面的键
keys.append((iterations-1))#把最后一次添加到迭代次数值列表中
# 依据历史数据绘制所有分割直线
for key in keys:
    weights, bias=perceptron.history[key] #从历史数据字典中获取选中的权重和阈值数据 
    if weights[1] != 0:
        x_vals = np.linspace(x_min, x_max,100)
        y_vals = -(weights[0] * x_vals - bias) / weights[1]
        random_rgb = np.random.rand(3)#生成三个0到1之间的随机数作为RGB分量
        color = mcolors.to_rgba(random_rgb) # 将RGB值转换为RGBA格式
        plt.plot(x_vals, y_vals, color=color, label=f'Iter {key+1}')
plt.xlabel('特征1')
plt.ylabel('特征2')
plt.title('感知机学习过程展示')
plt.legend()
plt.show()

### 代码8.15

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#阶跃函数
def step_function(x):
    return np.array([1 if i >=0 else 0 for i in x])
# Sigmoid函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
# ReLU函数
def relu(x):
    return np.maximum(0, x)
# Softmax函数
def softmax(x):
    if len(x.shape) > 1:
        x = x - np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    else:
        x = x - np.max(x)
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x)
#生成输入值
x = np.arange(-20.0, 20.0, 0.1)
#计算各激活函数的输出
y_step = step_function(x)
y_sigmoid = sigmoid(x)
y_relu = relu(x)
#设置中文显示和坐标轴负号
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams[
'axes.unicode_minus'] = False
# 绘制阶跃函数图像
plt.figure(figsize=(12, 8))
plt.subplot(1, 3, 1)
plt.plot(x, y_step)
plt.title('阶跃函数')
plt.ylim(-0.1, 1.1)
#绘制 Sigmoid 函数图像
plt.subplot(1, 3, 2)
plt.plot(x, y_sigmoid)
plt.title('Sigmoid函数')
plt.ylim(-0.1, 1.1)
#绘制 ReLU 函数图像
plt.subplot(1, 3, 3)
plt.plot(x, y_relu)
plt.title('ReLU函数')
plt.ylim(-0.1, 5.0)
plt.tight_layout()
plt.show()
#测试 Softmax 函数
test_vector = np.array([1, 2, 3, 4, 5])
softmax_output = softmax(test_vector)
print("Softmax 函数输入向量:", test_vector)
print("Softmax 函数输出概率分布:", softmax_output)
print("Softmax 函数输出概率之和:", np.sum(softmax_output))

### 代码8.16

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#定义sigmoid函数
def sigmoid(x):
    return 1/(1+np.exp(-x))
#定义线性可分逻辑门函数
def and_gate(x1, x2): #与门
    #线性计算： w1*x1 + w2*x2 + b 
    weight1, weight2, bias = 10, 10, -15 
    linear_output = weight1 * x1 + weight2 * x2 + bias
    sigmoid_out=round(sigmoid(linear_output))
    return sigmoid_out
def or_gate(x1, x2): #或门
    #线性计算： w1*x1 + w2*x2 + b 
    weight1, weight2, bias = 10, 10, -5 
    linear_output = weight1 * x1 + weight2 * x2 + bias
    sigmoid_out=round(sigmoid(linear_output))
    return sigmoid_out
def nand_gate(x1, x2): #与非门
    #线性计算：w1*x1 + w2*x2 + b 
    weight1, weight2, bias = -10, -10, 15 
    linear_output = weight1 * x1 + weight2 * x2 + bias
    sigmoid_out=round(sigmoid(linear_output))
    return sigmoid_out 
#生成两个测谎题目的所有可能答案组合
input_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
#计算三个逻辑门的输出
and_output = [and_gate(x[0], x[1]) for x in input_data]
or_output = [or_gate(x[0], x[1]) for x in input_data]
nand_output = [nand_gate(x[0], x[1]) for x in input_data]
outputs = [and_output, or_output, nand_output]
titles = ['与门分类结果', '或门分类结果', '与非门分类结果']
#创建子图
plt.rcParams['font.sans-serif'] = ['SimHei'] # 可根据系统情况选择其他中文字体，如 'Microsoft YaHei' 
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题 
fig, axes = plt.subplots(1, 3, figsize=(15, 10))
axes = axes.flatten()
# 用于存储图例句柄和标签
handles = []
labels = []
for i in range(len(outputs)):
    output = outputs[i]
    axes[i].scatter(input_data[np.array(output) ==0][:, 0], input_data[np.array(output) == 0][:, 1], c='r',label='输出0')
    axes[i].scatter(input_data[np.array(output) ==1][:, 0], input_data[np.array(output) == 1][:, 1], c='b',label='输出1')
    axes[i].set_title(titles[i])
    axes[i].set_xlabel('答案1')
    axes[i].set_ylabel('答案2')
    axes[i].set_xlim(-1, 2)
    axes[i].set_ylim(-1, 2)
   # 只在第一个子图中获取图例句柄和标签
    if i == 0:
        handles, labels = axes[i].get_legend_handles_labels()
# 在整个图形中添加一个图例
fig.legend(handles, labels, loc=2, bbox_to_anchor=(0.9, 0.85))
plt.tight_layout()
plt.show()

### 代码8.17

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#定义sigmoid函数
def sigmoid(x):
    return 1/(1+np.exp(-x))
#定义线性可分逻辑门函数
def and_gate(x1, x2): #与门
    #线性计算： w1*x1 + w2*x2 + b 
    weight1, weight2, bias = 10, 10, -15 
    linear_output = weight1 * x1 + weight2 * x2 + bias
    sigmoid_out=round(sigmoid(linear_output))
    return sigmoid_out
def or_gate(x1, x2): #或门
    #线性计算： w1*x1 + w2*x2 + b 
    weight1, weight2, bias = 10, 10, -5 
    linear_output = weight1 * x1 + weight2 * x2 + bias
    sigmoid_out=round(sigmoid(linear_output))
    return sigmoid_out
def nand_gate(x1, x2): #与非门
    #线性计算：w1*x1 + w2*x2 + b 
    weight1, weight2, bias = -10, -10, 15 
    linear_output = weight1 * x1 + weight2 * x2 + bias
    sigmoid_out=round(sigmoid(linear_output))
    return sigmoid_out
#通过线性可分逻辑门函数组合实现线性不可分逻辑门
def xor_gate(x1,x2): #异或门
    R1=or_gate(x1,x2) #通过"或门"获得R1
    R2=nand_gate(x1,x2) #通过"与非门"获得R2
    y=and_gate(R1,R2) #R1和R2通过"与门"获得y
    return y
#生成两个测谎题目的所有可能答案组合
input_data1 = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
#生成横轴和纵轴-1.1到2之间的矩形内100个平均分布的点
x = np.linspace(-1.1, 2, 10)
y = np.linspace(-1.1, 2, 10)
X, Y = np.meshgrid(x, y)
input_data2 = np.column_stack([X.flatten(), Y.flatten()])
# 通过异或门函数生成数据点的分类
output1 = [xor_gate(x[0], x[1]) for x in input_data1]
output2 = [xor_gate(x[0], x[1]) for x in input_data2]
#将inputdata2和output2数据合并，并输出到excel里，以备后续训练多层感知机使用
output2_arr=np.array(output2).reshape(len(output2),1)
data=np.column_stack((input_data2,output2_arr)) #合并输入和输出数据
np.random.shuffle(data)#随机打乱顺序
df=pd.DataFrame(data=data,columns=["输入1","输入2","输出"])
df.to_excel("异或门函数输入和输出数据.xlsx",index=False)
plt.rcParams['font.sans-serif'] = ['SimHei']#设置 matplotlib 支持中文
plt.rcParams['axes.unicode_minus'] = False
#创建包含两个子图的图形
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
#绘制 input_data1 的散点图
axes[0].scatter(input_data1[np.array(output1) == 0][:, 0], input_data1[np.array(output1) == 0][:, 1], c='r', label='输出0')
axes[0].scatter(input_data1[np.array(output1) == 1][:, 0], input_data1[np.array(output1) == 1][:, 1], c='b', label='输出1')
axes[0].set_title('input_data1 异或门分类结果')
axes[0].set_xlabel('答案1')
axes[0].set_ylabel('答案2')
axes[0].set_xlim(-1.1, 2)
axes[0].set_ylim(-1.1, 2)
#绘制 input_data2 的散点图
axes[1].scatter(input_data2[np.array(output2) == 0][:, 0], input_data2[np.array(output2) == 0][:, 1], c='r', label='输出0')
axes[1].scatter(input_data2[np.array(output2) == 1][:, 0], input_data2[np.array(output2) == 1][:, 1], c='b', label='输出1')
axes[1].set_title('input_data2 异或门分类结果')
axes[1].set_xlabel('答案1')
axes[1].set_ylabel('答案2')
axes[1].set_xlim(-1.1, 2)
axes[1].set_ylim(-1.1, 2)
#添加图例
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center',bbox_to_anchor=(0.4, 0.85))
plt.tight_layout()
plt.show()

### 代码8.18

In [ ]:
import numpy as np
def cross_entropy(y_true, y_pred):
    #防止对数计算出现无穷大
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    return -np.sum(y_true * np.log(y_pred))
#示例数据
# 理想硬币的概率分布
p= np.array([0.5,0.5])
#实验硬币的概率分布
q = np.array([0.6, 0.4])
ce = cross_entropy(p, q)
print(f"单个样本的交叉熵: {ce}")

### 代码8.19

In [ ]:
import numpy as np
# （1）定义计算函数
#sigmoid激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
#交叉熵损失函数
def cross_entropy_loss(y_true, y_pred):
    epsilon =1e-7 # 防止log(0)
    return -(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))
#（2）前向传播步骤
def forward_propagation(x_2,y_t,ω_21,ω_22,v_1_other_addend,v_2_other_addend):
    v_1 = ω_21 * x_2+v_1_other_addend   #步骤⑤：计算 v1
    R_2 = sigmoid(v_1)   #步骤④：计算 R_2
    v_2 = ω_22 * R_2+v_2_other_addend   #步骤③： 计算 v_2
    y = sigmoid(v_2)   #步骤②：计算 y    
    L = cross_entropy_loss(y_t, y)  #步骤①：计算交叉熵损失
    print("模型预测的交叉熵是：",L)
    return L,y,R_2,
#（3）反向传播步骤
def barkward_propagation(x_2,y_t,ω_22,R_2):
    dL_dy = -(y_t / y - (1 - y_t) / (1 - y)) # 步骤①：计算 ∂L/∂y
    dy_dv2 = y * (1 - y)  #步骤②：计算 ∂y/∂v_2
    dv2_dω22 = R_2  #步骤③：计算 ∂v_2/∂ω_22 和 ∂v_2/∂R_2
    dv2_dR2 = ω_22
    dR2_dv1 = R_2 * (1 - R_2)  #步骤④：计算 ∂R_2/∂v_1
    dv1_dω21 = x_2  #步骤⑤：计算 ∂v_1/∂ω_21
    #链式求导
    dL_dω22 = dL_dy * dy_dv2 * dv2_dω22
    dL_dω21 = dL_dy * dy_dv2 * dv2_dR2 * dR2_dv1 * dv1_dω21
    return dL_dω22,dL_dω21
#（4）设定输入数据
x_2 = 0.5 # 示例值，可根据实际情况修改
y_t = 1 # 真实标签值，示例值，可根据实际情况修改
ω_21 = 0.3 # 示例值，可根据实际情况修改
ω_22 = 0.4 # 示例值，可根据实际情况修改
v_1_other_addend=1.8 #代表输入值x_1和x_0的加权和
v_2_other_addend=1.2 #代表输入值R_1和R_0的加权和
#（5）执行前向传播和反向传播5次，观察交叉熵的变化情况
alpha=5 #学习率
for i in range(5):
    L,y,R_2=forward_propagation(x_2,y_t,ω_21,ω_22,v_1_other_addend,v_2_other_addend)  #前向传播计算
    dL_dω22,dL_dω21=barkward_propagation(x_2,y_t,ω_22,R_2) #反向传播计算
    ω_22=ω_22-alpha*dL_dω22  #利用梯度下降法更新ω_22
    ω_21=ω_21-alpha*dL_dω21  #利用梯度下降法更新ω_21
    print("第" + str(i + 1) + "次模型输出:")
    print("预测值y和交叉熵L：", y)
    print("预测值y和交叉熵L：", L)
    print("更新后的参数ω_21和ω_22：",np.round(ω_21,2) ,np.round(ω_22,2))

### 代码8.20

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#（1）定义前向计算和反向计算用到的函数
# sigmoid激活函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
#交叉熵损失函数
def cross_entropy_loss(y_true, y_pred):
    epsilon =1e-7 # 防止log(0)
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon)) / m
    return loss
#计算模型预测准确率
def calculate_accuracy(y_pred_proba, y_true):
    y_pred = np.where(y_pred_proba >=0.5, 1, 0)#模型输出大于等于0.5转化为1，否则为0。
    correct_count = np.sum(y_pred == y_true) #模型输出与真实标签相同的个数
    total_count = len(y_true) #总的样本数
    accuracy = correct_count / total_count if total_count > 0 else 0 #计算预测正确的比例
    return accuracy
#（2）创建多层感知机模型的类
class MultiLayerPerceptron:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.weights1 = np.random.rand(self.input_size + 1, self.hidden_size)
        self.weights2 = np.random.rand(self.hidden_size + 1, self.output_size)
        self.loss_and_accuracy_history={"loss":[],"accuracy":[]} #存放迭代过程中的历史数据
    def forward_propagation(self, X):#前向传播
        X_with_bias = np.hstack((np.ones((X.shape[0], 1)), X))
        self.hidden_layer = sigmoid(np.dot(X_with_bias, self.weights1))
        hidden_layer_with_bias = np.hstack((np.ones((self.hidden_layer.shape[0], 1)), self.hidden_layer))
        self.output_layer = sigmoid(np.dot(hidden_layer_with_bias, self.weights2))
        return self.output_layer
    def back_propagation(self, X, y, output):#反向传播
        m = y.shape[0]
       #计算交叉熵损失对输出层激活值的导数∂L/∂y
        d_loss_d_output = -(y / (output + 1e-7) - (1 - y) / (1 - output + 1e-7))
       #计算输出层激活值对输出层加权和的导数（sigmoid导数）∂y/∂v2
        d_output_d_v2 = output*(1-output)
       #输出层加权和对权重2的导数∂v2/∂w2
        d_v2_d_w2 = np.hstack((np.ones((self.hidden_layer.shape[0], 1)), self.hidden_layer)).T
        #通过链式求导计算输出层权重2的梯度∂L/∂w2
        d_loss_d_w2 = np.dot(d_v2_d_w2, d_output_d_v2 * d_loss_d_output) / m
        #计算隐藏层激活值对隐藏层加权和的导数∂R/∂v01
        d_hidden_d_v01 = self.hidden_layer*(1-self.hidden_layer)
        #隐藏层加权和对权重1的导数∂v01/∂w01
        d_v01_d_w01 = np.hstack((np.ones((X.shape[0], 1)), X)).T
        #计算输出层加权和对隐藏层激活值的导数∂v2/∂R
        d_v2_d_hidden = self.weights2[1:, :].T
        #通过链式求导计算隐藏层权重的梯度
        d_loss_d_w01 = np.dot(d_v01_d_w01, d_hidden_d_v01 * np.dot(d_output_d_v2 * d_loss_d_output, d_v2_d_hidden)) / m
        return d_loss_d_w01, d_loss_d_w2
#更新权重
    def update_weights(self, d_loss_d_w1, d_loss_d_w2, learning_rate):
        self.weights1 -= learning_rate * d_loss_d_w1
        self.weights2 -= learning_rate * d_loss_d_w2
#训练方法
    def fit(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward_propagation(X)
            loss = cross_entropy_loss(y_true=y, y_pred=output)
            accuracy=calculate_accuracy(y_true=y,y_pred_proba=output)
            d_loss_d_w1, d_loss_d_w2 = self.back_propagation(X, y, output)
            self.update_weights(d_loss_d_w1, d_loss_d_w2, learning_rate)
            self.loss_and_accuracy_history["loss"].append(loss) #存储每轮次loss值
            self.loss_and_accuracy_history["accuracy"].append(accuracy) #存储每次迭代的accuracy值
        return self.weights1, self.weights2
#（3）导入前期在异或门章节生成的数据，作为训练数据
data=pd.read_excel("异或门函数输入和输出数据.xlsx")
X=data[["输入1","输入2"]].values
y=data["输出"].values.reshape((len(data),1))
#（4）实例化多层感知机并进行4次不同随机种子训练，可视化损失函数和准确率对比训练效果
for i in range(4):
    np.random.seed(i)#创建多层感知机实例
    mlp = MultiLayerPerceptron(input_size=2, hidden_size=2, output_size=1)
   #训练模型
    weights1, weights2 = mlp.fit(X, y, epochs=10000, learning_rate=0.1)
    print("随机种子为"+str(i)+"的迭代结果：")
    print("①输入层到隐藏层的系数 ",weights1)
    print("②隐藏层到输入层的系数 ",weights2)
    plt.subplot(2,2,i+1)
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.plot(mlp.loss_and_accuracy_history["loss"], color="r", label="交叉熵")
    plt.plot(mlp.loss_and_accuracy_history["accuracy"], color="b", label="正确率")
    plt.xlabel("迭代次数")
plt.legend()
plt.show()

### 代码8.21

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import matplotlib.pyplot as plt
np.random.seed(123)
#（1）生成数据三个数据集合
#①模拟生成1000个真人作答数据
real_data = np.random.randint(0, 3, size=(1000, 20))
#使其中四个题的答题方向与另外16个题目相反
opposite_indices = np.random.choice(20, 4, replace=False) 
for i in range(1000):    
    real_data[i, opposite_indices] = 4 - real_data[i, opposite_indices]
#②生成随机作答数据
random_data=np.random.randint(0, 4, size=(500, 20))
#③生成直线作答数据
value = np.random.randint(0, 4)
straight_data=np.ones((500, 20)) * value
#④合并数据集并打乱顺序
X = np.vstack((real_data, random_data, straight_data))#合并量表得分数据
y = np.array([1] * 1000 + [2] *500 + [3] * 500) #生成标签数据
indices = np.arange(X.shape[0])  #生成一个与总数据集长度的索引数组
np.random.shuffle(indices)#把索引数组随机打乱
X = X[indices]#用打乱后的索引数值给X数组排序
y = y[indices]#用打乱后的索引数值给y数组排序
#（2）训练模型并记录过程指标
#y①划分训练数据和测试数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#②设置模型参数
mlp = MLPClassifier(hidden_layer_sizes=(30,), max_iter=100,  random_state=123)
train_loss_history = []
train_acc_history = []
test_loss_history = []
test_acc_history = []
for epoch in range(100):
    mlp.partial_fit(X_train, y_train, classes=np.unique(y))  #训练模型启动
    #记录训练集的交叉熵和准确率
    y_pred_train = mlp.predict_proba(X_train)
    train_loss = log_loss(y_train, y_pred_train)
    train_acc = accuracy_score(y_train, mlp.predict(X_train))
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    #记录测试集的交叉熵和准确率
    y_pred_test = mlp.predict_proba(X_test)
    test_loss = log_loss(y_test, y_pred_test)
    test_acc = accuracy_score(y_test, mlp.predict(X_test))
    test_loss_history.append(test_loss)
    test_acc_history.append(test_acc)
#（3）可视化模型的训练结果
plt.rcParams['font.sans-serif'] = ['SimHei']#设置图片中文显示
plt.rcParams['axes.unicode_minus'] = False
plt.subplot(1, 2, 1)#创建第一张子图
plt.plot(train_loss_history,color="r",label="训练数据交叉熵")
plt.plot(test_loss_history, color="b",label="测试数据交叉熵")
plt.xlabel('训练轮数')
plt.ylabel('交叉熵')
plt.legend()
plt.subplot(1, 2, 2)#创建第二张子图
plt.plot(train_acc_history, color="r", label="训练数据准确率")
plt.plot(test_acc_history, color="b", label="测试数据准确率")
plt.xlabel('训练轮数')
plt.ylabel('准确率')
plt.legend()
plt.tight_layout()
plt.show()